In [6]:
def etude_fichier(df):

    print("Nombre de colonnes :", df.shape[1])
    print()
    print("Le type est : \n", df.dtypes)
    print()
    print('Nombre de valeurs uniques :')
    print(df.nunique())
    print()
    print('Le nombre de valeurs manquantes :\n', df.isnull().sum())

In [5]:
def analyse_statistique(df):

    stats = df.describe(include='all')
    stats.loc['median'] = df.median()
    stats.loc['range'] = df.max() - df.min()
    stats.loc['skewness'] = df.skew()
    stats.loc['kurtosis'] = df.kurtosis()

    return stats

In [4]:
import pandas as pd
import numpy as np

def detecter_outliers(df, column, seuil=2.0):
    """
    Détecte les outliers dans une colonne d'un DataFrame en utilisant le Z-score.

    Args:
        df (pandas.DataFrame): Le DataFrame contenant les données numériques.
        column (str): Le nom de la colonne à étudier.
        seuil (float): Le seuil pour définir les outliers. Par défaut, 2.0.

    Returns:
        float: Le pourcentage d'outliers dans la colonne spécifiée.
    """
    if column not in df.columns:
        raise ValueError(f"La colonne '{column}' n'existe pas dans le DataFrame.")
    
    series = df[column]
    z_scores = np.abs((series - series.mean()) / series.std())
    outliers_mask = z_scores > seuil
    outliers_percentage = (outliers_mask.mean() * 100).round(2)
    return outliers_percentage

# Exemple d'utilisation :
# Supposons que vous ayez un DataFrame df avec des colonnes numériques
# Vous pouvez détecter le pourcentage d'outliers pour la colonne 'colonne1' comme suit :
pourcentage_outliers_colonne1 = detecter_outliers(df, column='colonne_1', seuil=2.0)
print(pourcentage_outliers_colonne1)

In [2]:
def analyse_cles(df):
    """
    Analyse les colonnes du DataFrame pour détecter les clés primaires et les clés étrangères potentielles.

    Args:
        df (pandas.DataFrame): Le DataFrame à analyser.

    Returns:
        dict: Un dictionnaire contenant deux listes, l'une pour les clés primaires et l'autre pour les clés étrangères potentielles.
    """
    primary_keys = []
    foreign_keys = []

    for column in df.columns:
        unique_count = df[column].nunique()
        total_count = df[column].count()

        if unique_count == total_count:
            primary_keys.append(column)
        elif unique_count > 1 and unique_count < total_count:
            foreign_keys.append(column)

    return {"primary_keys": primary_keys, "foreign_keys": foreign_keys}


In [3]:
def identifier_differences_entre_cles(df1, df2, keys=None):
    """
    Identifie les différences entre les clés (clés primaires ou colonnes spécifiées) de deux DataFrames.

    Args:
        df1 (pandas.DataFrame): Le premier DataFrame.
        df2 (pandas.DataFrame): Le deuxième DataFrame.
        keys (list or str, optional): Les clés (nom des colonnes) à utiliser pour l'identification des différences. 
                                     Si None, utilise les clés primaires (colonnes ayant le même nom dans les deux DataFrames).
                                     Par défaut, None.

    Returns:
        dict: Un dictionnaire contenant deux listes, l'une pour les clés présentes uniquement dans df1,
              et l'autre pour les clés présentes uniquement dans df2.
    """
    if keys is None:
        keys = list(df1.columns.intersection(df2.columns))

    unique_keys_df1 = set(df1[keys].drop_duplicates().apply(tuple, axis=1))
    unique_keys_df2 = set(df2[keys].drop_duplicates().apply(tuple, axis=1))

    keys_only_in_df1 = unique_keys_df1 - unique_keys_df2
    keys_only_in_df2 = unique_keys_df2 - unique_keys_df1

    return {"keys_only_in_df1": list(keys_only_in_df1), "keys_only_in_df2": list(keys_only_in_df2)}


In [8]:
def traiter_valeurs_manquantes(df, method='mean', columns=None):
    """
    Remplace les valeurs manquantes du DataFrame en utilisant différentes méthodes.

    Args:
        df (pandas.DataFrame): Le DataFrame à traiter.
        method (str): La méthode de remplacement des valeurs manquantes. 
                      'mean' pour la moyenne, 'median' pour la médiane, 'mode' pour le mode, etc.
        columns (list or str, optional): La (ou les) colonne(s) spécifique(s) sur laquelle (lesquelles) appliquer le traitement.
                                         Si None, applique le traitement à toutes les colonnes. Par défaut, None.

    Returns:
        pandas.DataFrame: Le DataFrame avec les valeurs manquantes traitées.
    """
    if method == 'mean':
        if columns is None:
            return df.fillna(df.mean())
        else:
            return df.fillna(df.mean()[columns])
    elif method == 'median':
        if columns is None:
            return df.fillna(df.median())
        else:
            return df.fillna(df.median()[columns])
    elif method == 'mode':
        if columns is None:
            return df.fillna(df.mode().iloc[0])
        else:
            return df.fillna(df.mode().iloc[0][columns])
    else:
        return df.fillna(method=method)

# Exemple d'utilisation :
# Supposons que vous ayez un DataFrame df avec une colonne 'colonne1' que vous voulez traiter les valeurs manquantes avec la moyenne :
# df_traite = traiter_valeurs_manquantes(df, method='mean', columns='colonne1')


In [7]:
def encoder_variables_categorielles(df, columns_to_encode):
    """
    Encode les variables catégorielles du DataFrame en utilisant la technique de "one-hot encoding".

    Args:
        df (pandas.DataFrame): Le DataFrame contenant les variables catégorielles à encoder.
        columns_to_encode (list): Liste des noms de colonnes catégorielles à encoder.

    Returns:
        pandas.DataFrame: Le DataFrame avec les variables catégorielles encodées.
    """
    return pd.get_dummies(df, columns=columns_to_encode)


In [9]:
from sklearn.linear_model import LinearRegression

def regression_lineaire(X, y):
    """
    Effectue une régression linéaire simple.

    Args:
        X (numpy.ndarray): Les caractéristiques (variables indépendantes).
        y (numpy.ndarray): La variable cible (variable dépendante).

    Returns:
        sklearn.linear_model.LinearRegression: Le modèle de régression linéaire entraîné.
    """
    model = LinearRegression()
    model.fit(X, y)
    return model


In [10]:
from sklearn.model_selection import train_test_split

def separation_entrainement_test(X, y, test_size=0.2, random_state=None):
    """
    Sépare les données en ensembles d'entraînement et de test.

    Args:
        X (numpy.ndarray): Les caractéristiques (variables indépendantes).
        y (numpy.ndarray): La variable cible (variable dépendante).
        test_size (float, optional): La proportion des données à inclure dans l'ensemble de test. Par défaut, 0.2.
        random_state (int, optional): La graine aléatoire pour la reproductibilité. Par défaut, None.

    Returns:
        tuple: Un tuple contenant les ensembles d'entraînement et de test pour X et y.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


In [11]:
import pandas as pd

def etudier_jointures(df1, df2, on_column):
    """
    Effectue une étude des jointures possibles entre deux DataFrames en utilisant la colonne spécifiée comme clé de jointure.

    Args:
        df1 (pandas.DataFrame): Le premier DataFrame.
        df2 (pandas.DataFrame): Le deuxième DataFrame.
        on_column (str): Le nom de la colonne à utiliser comme clé de jointure.

    Returns:
        dict: Un dictionnaire contenant les résultats de chaque jointure (inner, left, right, outer) 
              et une analyse des clés résultantes.
    """
    results = {}

    # Jointure interne (inner join)
    results['inner_join'] = pd.merge(df1, df2, how='inner', on=on_column)
    results['inner_join_analysis'] = analyse_cles_jointure(results['inner_join'], on_column)

    # Jointure à gauche (left join)
    results['left_join'] = pd.merge(df1, df2, how='left', on=on_column)
    results['left_join_analysis'] = analyse_cles_jointure(results['left_join'], on_column)

    # Jointure à droite (right join)
    results['right_join'] = pd.merge(df1, df2, how='right', on=on_column)
    results['right_join_analysis'] = analyse_cles_jointure(results['right_join'], on_column)

    # Jointure externe complète (full outer join)
    results['outer_join'] = pd.merge(df1, df2, how='outer', on=on_column)
    results['outer_join_analysis'] = analyse_cles_jointure(results['outer_join'], on_column)

    return results

def analyse_cles_jointure(df, on_column):
    """
    Analyse les clés résultantes d'une jointure.

    Args:
        df (pandas.DataFrame): Le DataFrame résultant de la jointure.
        on_column (str): Le nom de la colonne utilisée comme clé de jointure.

    Returns:
        dict: Un dictionnaire contenant le nombre de clés uniques avant et après la jointure.
    """
    unique_keys_before = df[on_column].nunique()
    unique_keys_after = df.drop_duplicates(subset=on_column)[on_column].nunique()

    return {"unique_keys_before": unique_keys_before, "unique_keys_after": unique_keys_after}
